# Urdu & Pashto Tokenization Notebook

This notebook demonstrates whitespace vs regex-based tokenization for Urdu text using the column **News Text**.

In [14]:
!pip install transformers

In [15]:
import pandas as pd
import re
import sentencepiece as spm
import os
from transformers import AutoTokenizer

In [16]:
!git clone https://github.com/anoopkunchukuttan/indic_nlp_library.git
!pip install ./indic_nlp_library


Cloning into 'indic_nlp_library'...
remote: Enumerating objects: 1404, done.
remote: Counting objects: 100% (185/185), done.
remote: Compressing objects: 100% (58/58), done.
remote: Total 1404 (delta 139), reused 152 (delta 124), pack-reused 1219 (from 1)
Receiving objects: 100% (1404/1404), 9.57 MiB | 12.21 MiB/s, done.
Resolving deltas: 100% (749/749), done.
Processing ./indic_nlp_library
  Preparing metadata (setup.py) ... done
  Created wheel for indic_nlp_library: filename=indic_nlp_library-0.92-py3-none-any.whl size=40778 sha256=09ea68da50f50894e8a52ef8f4d030b7e5b842268dae9120cd4f3a979daa082a
  Stored in directory: /tmp/pip-ephem-wheel-cache-g53vugu7/wheels/7e/4b/68/314b55f016ac49a6dbc7c026d2744cc8783fc7cb1a845d3248
Successfully built indic_nlp_library


In [17]:
!git clone https://github.com/anoopkunchukuttan/indic_nlp_resources.git /content/indic_nlp_resources

Cloning into '/content/indic_nlp_resources'...
remote: Enumerating objects: 139, done.
remote: Counting objects: 100% (13/13), done.
remote: Compressing objects: 100% (13/13), done.
remote: Total 139 (delta 2), reused 2 (delta 0), pack-reused 126 (from 1)
Receiving objects: 100% (139/139), 149.77 MiB | 32.41 MiB/s, done.
Resolving deltas: 100% (53/53), done.
Updating files: 100% (28/28), done.


In [18]:
from indicnlp.tokenize import indic_tokenize

In [19]:
import os

os.listdir("/kaggle/input")


['urdu-dataset']

In [20]:
import os

DATA_PATH = "/kaggle/input/urdu-dataset"

print(os.listdir(DATA_PATH))


['UrduDataset.xlsx']


In [21]:

import pandas as pd

file_path = "/kaggle/input/urdu-dataset/UrduDataset.xlsx"

try:
    df = pd.read_excel(file_path)
    print("Dataset loaded successfully:")

except FileNotFoundError:
    print(f"Error: The file at '{file_path}' was not found.")
except Exception as e:
    print(f"An error occurred: {e}")


Dataset loaded successfully:


In [22]:

print(df.columns)


Index(['Index', 'Headline', 'News Text', 'Category', 'Date', 'URL', 'Source',
       'News length'],
      dtype='object')


## Whitespace Tokenization (Not Recommended)

In [23]:

df['whitespace_tokens'] = df['News Text'].astype(str).apply(lambda x: x.split())
df[['News Text', 'whitespace_tokens']].head()


,News Text,whitespace_tokens
0,اسلام باد عالمی بینک خیبرپختونخوا کے قبائلی اض...,"[اسلام, باد, عالمی, بینک, خیبرپختونخوا, کے, قب..."
1,اسلام باد فیڈرل بورڈ ریونیو ایف بی نے دسمبر کی...,"[اسلام, باد, فیڈرل, بورڈ, ریونیو, ایف, بی, نے,..."
2,اسلام باد بورڈ انویسٹمنٹ بی او ئی کے چیئرمین ع...,"[اسلام, باد, بورڈ, انویسٹمنٹ, بی, او, ئی, کے, ..."
3,اسلام اباد پاکستان میں ماہ نومبر میں مسلسل تیس...,"[اسلام, اباد, پاکستان, میں, ماہ, نومبر, میں, م..."
4,اسلام باد نیشنل ٹرانسمیشن اینڈ ڈسپیچ کمپنی این...,"[اسلام, باد, نیشنل, ٹرانسمیشن, اینڈ, ڈسپیچ, کم..."


## Regex-based Unicode Tokenization (Recommended)

In [24]:

import re

def regex_tokenize(text):
    pattern = r'[\u0600-\u06FF]+'
    return re.findall(pattern, str(text))

df['regex_tokens'] = df['News Text'].apply(regex_tokenize)
df[['News Text', 'regex_tokens']].head()


,News Text,regex_tokens
0,اسلام باد عالمی بینک خیبرپختونخوا کے قبائلی اض...,"[اسلام, باد, عالمی, بینک, خیبرپختونخوا, کے, قب..."
1,اسلام باد فیڈرل بورڈ ریونیو ایف بی نے دسمبر کی...,"[اسلام, باد, فیڈرل, بورڈ, ریونیو, ایف, بی, نے,..."
2,اسلام باد بورڈ انویسٹمنٹ بی او ئی کے چیئرمین ع...,"[اسلام, باد, بورڈ, انویسٹمنٹ, بی, او, ئی, کے, ..."
3,اسلام اباد پاکستان میں ماہ نومبر میں مسلسل تیس...,"[اسلام, اباد, پاکستان, میں, ماہ, نومبر, میں, م..."
4,اسلام باد نیشنل ٹرانسمیشن اینڈ ڈسپیچ کمپنی این...,"[اسلام, باد, نیشنل, ٹرانسمیشن, اینڈ, ڈسپیچ, کم..."


# **Indic NLP tokenization**

In [25]:
df['indic_tokens'] = df['News Text'].astype(str).apply(lambda x: indic_tokenize.trivial_tokenize(x))
df['indic_tokens']

0      [اسلام, باد, عالمی, بینک, خیبرپختونخوا, کے, قب...
1      [اسلام, باد, فیڈرل, بورڈ, ریونیو, ایف, بی, نے,...
2      [اسلام, باد, بورڈ, انویسٹمنٹ, بی, او, ئی, کے, ...
3      [اسلام, اباد, پاکستان, میں, ماہ, نومبر, میں, م...
4      [اسلام, باد, نیشنل, ٹرانسمیشن, اینڈ, ڈسپیچ, کم...
                             ...                        
194    [کراچی, سوئی, سدرن, گیس, کمپنی, ایس, ایس, جی, ...
195    [پیٹرولیم, ڈویژن, کے, شدید, احتجاج, کے, باوجود...
196    [اسلام, باد, کے, الیکٹرک, کے, لیے, بجلی, کی, ق...
197    [کابینہ, کی, اقتصادی, رابطہ, کمیٹی, ای, سی, سی...
198    [اسلام, باد, حکومت, نے, اسمگلنگ, پر, قابو, پان...
Name: indic_tokens, Length: 199, dtype: object

# Tokenize Urdu using BERT tokenizer

In [26]:
tokenizer = AutoTokenizer.from_pretrained("bert-base-multilingual-cased")
df['bert_encoding'] = df['News Text'].astype(str).apply(
    lambda x: tokenizer(
        x,
        truncation=True,
        max_length=512,
        padding=False,
        return_attention_mask=False,
        return_token_type_ids=False
    )
)

# Extract tokens safely
df['bert_tokens'] = df['bert_encoding'].apply(lambda x: tokenizer.convert_ids_to_tokens(x['input_ids']))
df['bert_len'] = df['bert_tokens'].apply(len)



tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/625 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

**Calculate The Tokens**

In [27]:
# 3. Calculate token lengths
# -------------------------------
df['whitespace_len'] = df['whitespace_tokens'].apply(len)
df['bert_len'] = df['bert_tokens'].apply(len)
df['indic_len'] = df['indic_tokens'].apply(len)
df['regex_len'] = df['regex_tokens'].apply(len)

## Token Length Comparison

In [28]:
df[['News Text', 'whitespace_len', 'regex_len', 'indic_len','bert_len']].head()

,News Text,whitespace_len,regex_len,indic_len,bert_len
0,اسلام باد عالمی بینک خیبرپختونخوا کے قبائلی اض...,375,370,375,512
1,اسلام باد فیڈرل بورڈ ریونیو ایف بی نے دسمبر کی...,429,412,429,512
2,اسلام باد بورڈ انویسٹمنٹ بی او ئی کے چیئرمین ع...,446,445,446,512
3,اسلام اباد پاکستان میں ماہ نومبر میں مسلسل تیس...,490,446,490,512
4,اسلام باد نیشنل ٹرانسمیشن اینڈ ڈسپیچ کمپنی این...,588,576,588,512


In [29]:
# 4. Preview tokenized dataset
# -------------------------------
df[['News Text', 'whitespace_tokens', 'regex_tokens', 'indic_tokens', 'bert_tokens']].head()

,News Text,whitespace_tokens,regex_tokens,indic_tokens,bert_tokens
0,اسلام باد عالمی بینک خیبرپختونخوا کے قبائلی اض...,"[اسلام, باد, عالمی, بینک, خیبرپختونخوا, کے, قب...","[اسلام, باد, عالمی, بینک, خیبرپختونخوا, کے, قب...","[اسلام, باد, عالمی, بینک, خیبرپختونخوا, کے, قب...","[[CLS], اسلام, با, ##د, عالمی, بین, ##ک, خ, ##..."
1,اسلام باد فیڈرل بورڈ ریونیو ایف بی نے دسمبر کی...,"[اسلام, باد, فیڈرل, بورڈ, ریونیو, ایف, بی, نے,...","[اسلام, باد, فیڈرل, بورڈ, ریونیو, ایف, بی, نے,...","[اسلام, باد, فیڈرل, بورڈ, ریونیو, ایف, بی, نے,...","[[CLS], اسلام, با, ##د, فی, ##ڈر, ##ل, بو, ##ر..."
2,اسلام باد بورڈ انویسٹمنٹ بی او ئی کے چیئرمین ع...,"[اسلام, باد, بورڈ, انویسٹمنٹ, بی, او, ئی, کے, ...","[اسلام, باد, بورڈ, انویسٹمنٹ, بی, او, ئی, کے, ...","[اسلام, باد, بورڈ, انویسٹمنٹ, بی, او, ئی, کے, ...","[[CLS], اسلام, با, ##د, بو, ##ر, ##ڈ, ان, ##وی..."
3,اسلام اباد پاکستان میں ماہ نومبر میں مسلسل تیس...,"[اسلام, اباد, پاکستان, میں, ماہ, نومبر, میں, م...","[اسلام, اباد, پاکستان, میں, ماہ, نومبر, میں, م...","[اسلام, اباد, پاکستان, میں, ماہ, نومبر, میں, م...","[[CLS], اسلام, اب, ##اد, پاکستان, میں, ما, ##ہ..."
4,اسلام باد نیشنل ٹرانسمیشن اینڈ ڈسپیچ کمپنی این...,"[اسلام, باد, نیشنل, ٹرانسمیشن, اینڈ, ڈسپیچ, کم...","[اسلام, باد, نیشنل, ٹرانسمیشن, اینڈ, ڈسپیچ, کم...","[اسلام, باد, نیشنل, ٹرانسمیشن, اینڈ, ڈسپیچ, کم...","[[CLS], اسلام, با, ##د, ن, ##یش, ##نل, ٹ, ##را..."


## Save Tokenized Dataset

In [35]:
output_path = "/kaggle/working/UrduDataset_Tokenized.xlsx"
df.to_excel(output_path, index=False)

print("Tokenized dataset saved successfully!")


Tokenized dataset saved successfully!
